In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm

# Load data
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')

# Utils
clienti = list(ordini_per_cliente.keys())
tot = sum(tot_per_cliente.values())
norm = {k: v/tot for k,v in tot_per_cliente.items()}

In [268]:
### Generate synthetic data

## params
num_samples = 15        # Number of samples to generate

# Set to True to generate data starting from today (orders in entrata oggi)
today = True        

# Altrimenti imposta un range
start = '2024-10-01'
end = '2024-12-31'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))

    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    print(rows)
    if rows > 0:
        articoli = quantity[quantity['Cliente'] == cliente]
        
        for _ in range(num + 1):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']
            quantity_mean = art['mean']
            quantity_std = art['std']
            giorni = art['mean_differenza_giorni']

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:
                kg = quantity_mean
            
            if today:
                data_inizio = pd.Timestamp.today().date()
            else:
                date_range = pd.date_range(start=start, end=end)
                data_inizio = np.random.choice(date_range)
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni)
            data_fine = pd.Timestamp(data_fine).date()

            print(f"Cliente: {cliente}, Articolo: {articolo}, Quantity: {kg}, Data inizio: {data_inizio}, Data fine: {data_fine}")

            # Append the generated data to new_data list
            new_data.append({
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

# Create a DataFrame from the new_data list
columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)
res

Generating data: 100%|██████████| 15/15 [00:00<00:00, 1682.70it/s]

2
Cliente: FROHN GMBH, Articolo: S14-60005V2DN, Quantity: 39.0, Data inizio: 2024-10-22, Data fine: 2024-11-17
4
Cliente: APPLE, Articolo: S18-60042V4DN, Quantity: 26.0, Data inizio: 2024-10-22, Data fine: 2024-11-17
3
Cliente: CALZIFICIO ATHENA, Articolo: X22-7810S-ZN354, Quantity: 150.0, Data inizio: 2024-10-22, Data fine: 2024-10-30
2
Cliente: CALZIFICIO C&LD, Articolo: 2269ZF2, Quantity: 22.0, Data inizio: 2024-10-22, Data fine: 2024-10-25
6
Cliente: CALZIFICIO NUOVO GIODI, Articolo: 2246ZN 13500, Quantity: 58.0, Data inizio: 2024-10-22, Data fine: 2024-11-17
Cliente: CALZIFICIO NUOVO GIODI, Articolo: 1724S+ZN, Quantity: 73.0, Data inizio: 2024-10-22, Data fine: 2024-11-17
1
Cliente: CALZIFICIO LUISA, Articolo: 2246ZN 13500, Quantity: 1200.0, Data inizio: 2024-10-22, Data fine: 2024-11-17
1
Cliente: GT 2000, Articolo: 2218ZN, Quantity: 414.0, Data inizio: 2024-10-22, Data fine: 2024-10-23
2
Cliente: CALZIFICIO ZETA, Articolo: 33002DN, Quantity: 100.0, Data inizio: 2024-10-22, Data 

,cliente,cod_articolo,quantity,data inserimento,data consegna
0,FROHN GMBH,S14-60005V2DN,39.0,2024-10-22,2024-11-17
1,APPLE,S18-60042V4DN,26.0,2024-10-22,2024-11-17
2,CALZIFICIO ATHENA,X22-7810S-ZN354,150.0,2024-10-22,2024-10-30
3,CALZIFICIO C&LD,2269ZF2,22.0,2024-10-22,2024-10-25
4,CALZIFICIO NUOVO GIODI,2246ZN 13500,58.0,2024-10-22,2024-11-17
5,CALZIFICIO NUOVO GIODI,1724S+ZN,73.0,2024-10-22,2024-11-17
6,CALZIFICIO LUISA,2246ZN 13500,1200.0,2024-10-22,2024-11-17
7,GT 2000,2218ZN,414.0,2024-10-22,2024-10-23
8,CALZIFICIO ZETA,33002DN,100.0,2024-10-22,2024-11-17
9,CALZIFICIO GENNY,X23-7892DN mini,128.0,2024-10-22,2024-11-17
